In [2]:
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,BatchNormalization

In [3]:
import pickle
with open('EXP1.pickle', 'rb') as f:
    data = pickle.load(f)

In [4]:
data

[array([[14.18278985, 24.85084955, 32.26250018, ..., 10.00417662,
          7.46888704,  3.62294237],
        [12.57967283, 23.76741535, 30.37286871, ..., 14.95723319,
         11.13698112,  5.84942905],
        [13.22193417, 24.55529474, 30.39580222, ..., 13.83155641,
         10.59001351,  5.29500675],
        ...,
        [13.29084536, 24.17527752, 31.31167992, ..., 14.23324907,
         10.53464287,  4.58407225],
        [12.95507451, 24.79821619, 31.58751678, ..., 15.15993757,
         11.82398723,  5.91199362],
        [12.83678016, 22.97733591, 30.31822964, ...,  9.57554661,
          7.2255416 ,  3.23136044]]),
 array([ 3, 14,  9, ...,  4, 14, 15]),
 array([[13.28076564, 22.61852359, 31.40274702, ..., 10.60715627,
          7.27644917,  3.63822458],
        [12.94003695, 25.86111181, 29.316574  , ..., 10.92564205,
          8.02663748,  4.04376155],
        [14.43351816, 25.80163901, 33.27687417, ..., 11.81740281,
          9.40164209,  3.67045161],
        ...,
        [ 9.313

In [5]:
data[0].shape

(6400, 120)

In [6]:
import pandas as pd

df = pd.DataFrame(data[0], columns=[f"Feature_{i}" for i in range(1, 121)])
label = pd.DataFrame(data[1], columns=['Titik'])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df, label, test_size=0.3, random_state=142, stratify=label)

In [8]:
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback

class MyCallback(Callback):
    def __init__(self, threshold=0.80):
        super(MyCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get('val_sparse_categorical_accuracy')
        if val_accuracy is not None and val_accuracy >= self.threshold:
            print(f"\nAkurasi telah mencapai {self.threshold*100}% di epoch {epoch}, menghentikan pelatihan...")
            self.model.stop_training = True

# class MyCallback(Callback):
#     def __init__(self, threshold=0.80):
#         super(MyCallback, self).__init__()
#         self.threshold = threshold

#     def on_epoch_end(self, epoch, logs=None):
#         # Cek apakah akurasi validasi telah melebihi threshold
#         if logs.get('val_accuracy') >= self.threshold:
#             print(f"\nAkurasi telah mencapai {self.threshold*100}% di epoch {epoch}, menghentikan pelatihan...")
#             self.model.stop_training = True

# Membuat instance dari MyCallback
my_callback = MyCallback(threshold=0.94)

In [10]:
model = Sequential()
# LSTM
model.add(LSTM(256, return_sequences=True, input_shape=(120, 1)))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='softmax'))


In [11]:
import neural_structured_learning as nsl

In [12]:
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)

In [13]:
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [14]:
adv_model.fit({'feature': X_train, 'label': y_train}, validation_data={'feature': X_val, 'label': y_val}, batch_size=128, epochs=100,callbacks=[my_callback])

Epoch 1/100


35/35 [==============================] - 37s 790ms/step - loss: 2.5765 - sparse_categorical_crossentropy: 2.1454 - sparse_categorical_accuracy: 0.3127 - scaled_adversarial_loss: 0.4311 - val_loss: 3.2710 - val_sparse_categorical_crossentropy: 2.7257 - val_sparse_categorical_accuracy: 0.0792 - val_scaled_adversarial_loss: 0.5453
Epoch 2/100
35/35 [==============================] - 20s 566ms/step - loss: 1.7270 - sparse_categorical_crossentropy: 1.4336 - sparse_categorical_accuracy: 0.5462 - scaled_adversarial_loss: 0.2933 - val_loss: 3.2390 - val_sparse_categorical_crossentropy: 2.6986 - val_sparse_categorical_accuracy: 0.1245 - val_scaled_adversarial_loss: 0.5404
Epoch 3/100
35/35 [==============================] - 18s 524ms/step - loss: 1.2144 - sparse_categorical_crossentropy: 1.0090 - sparse_categorical_accuracy: 0.6842 - scaled_adversarial_loss: 0.2054 - val_loss: 2.5995 - val_sparse_categorical_crossentropy: 2.1648 - val_sparse_categorical_accuracy: 0.2677 - val_scaled_adversarial

In [15]:
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import TensorFlowV2Classifier

c:\Users\salma\anaconda3\envs\gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\salma\anaconda3\envs\gpu\lib\site-packages\art\estimators\certification\__init__.py:29: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [20]:
classifier = TensorFlowV2Classifier(
    model=adv_model,  # model asli, bukan adv_model
    nb_classes=len(np.unique(y_train)),
    input_shape=(len(X_train[0]),),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    clip_values=(X_val.min(), X_val.max())
)

# Kemudian, buat serangan PGD dan terapkan pada X_test
attack = ProjectedGradientDescent(classifier, eps=0.2, eps_step=0.01, max_iter=40)
x_test_adv = attack.generate(X_val)

# Evaluasi model yang belum dibungkus pada contoh adversarial yang dihasilkan
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Akurasi pada contoh adversarial: {accuracy*100:.2f}%')

TypeError: Exception encountered when calling layer "AdversarialRegularization" "                 f"(type AdversarialRegularization).

Cannot convert 'label' to EagerTensor of dtype float

Call arguments received by layer "AdversarialRegularization" "                 f"(type AdversarialRegularization):
  • inputs=tf.Tensor(shape=(32, 120), dtype=float32)
  • kwargs={'training': 'False'}